# 水平集函数的重置化
scipy.ndimage.distance_transform_edt 函数是用于计算每个非零元素到最近零元素的欧几里得距离的变换。

## (1) reinit 函数逐行解析

In [84]:
import numpy as np
from scipy import ndimage

def reinit_detailed(sampling, struc):
    """
    根据给定的结构重置化水平集函数

    该函数通过添加 void 单元的边界来扩展输入结构，计算到最近的 solid 和 void 单元
    的欧几里得距离，并计算水平集函数，该函数在 solid phase 内为正，在 void phase 中为负

    Parameters:
    - sampling: 距离尺度
    - struc ( ndarray - (nely, nelx) ): 表示结构的 solid(1) 和 void(0) 单元

    Returns:
    - lsf ( ndarray - (nely+2, nelx+2) ): 表示重置化后的水平集函数

    Notes:
    只适用于正方形单元
    """

    strucFull = np.zeros((nely + 2, nelx + 2))
    strucFull[1:-1, 1:-1] = struc
    print("strucFull:", strucFull.shape, "\n", strucFull)

    # Compute the distance to the nearest void (0-valued) cells.
    dist_to_0 = ndimage.distance_transform_edt(strucFull, sampling=sampling)
    print("dist_to_0:", dist_to_0.shape, "\n", dist_to_0)

    # Compute the distance to the nearest solid (1-valued) cells.
    print("(1-strucFull):", (1-strucFull).shape, "\n", 1-strucFull)
    dist_to_1 = ndimage.distance_transform_edt(1 - strucFull, sampling=sampling)
    print("dist_to_1:", dist_to_1.shape, "\n", dist_to_1)

    # Offset the distances by 0.5 to center the level set function on the boundaries.
    element_length = sampling / 2
    temp_0 = dist_to_0 - element_length
    print("temp0:", temp_0.shape, "\n", temp_0)
    temp_1 = dist_to_1 - element_length
    print("temp1:", temp_1.shape, "\n", temp_1)

    # Calculate the level set function, ensuring the correct sign inside and outside the structure.
    lsf = -(1 - strucFull) * temp_1 + strucFull * temp_0
    print("lsf", lsf.shape, "\n", lsf)

    return lsf

# 测试参数
nelx, nely = 6, 4
domain = [0, 6, 0, 4]
ew = (domain[1]-domain[0]) / nelx 
eh = (domain[3]-domain[2]) / nely 

# 创建一个测试用的结构数组
struc = np.ones((nely, nelx))

# 调用 reinit 函数
lsf = reinit_detailed(sampling=ew, struc=struc)

strucFull: (6, 8) 
 [[0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 1. 1. 1. 1. 1. 0.]
 [0. 1. 1. 1. 1. 1. 1. 0.]
 [0. 1. 1. 1. 1. 1. 1. 0.]
 [0. 1. 1. 1. 1. 1. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]]
dist_to_0: (6, 8) 
 [[0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 1. 1. 1. 1. 1. 0.]
 [0. 1. 2. 2. 2. 2. 1. 0.]
 [0. 1. 2. 2. 2. 2. 1. 0.]
 [0. 1. 1. 1. 1. 1. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]]
(1-strucFull): (6, 8) 
 [[1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 0. 0. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 0. 0. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1.]]
dist_to_1: (6, 8) 
 [[1.41421356 1.         1.         1.         1.         1.
  1.         1.41421356]
 [1.         0.         0.         0.         0.         0.
  0.         1.        ]
 [1.         0.         0.         0.         0.         0.
  0.         1.        ]
 [1.         0.         0.         0.         0.         0.
  0.         1.        ]
 [1.         0.         0.         0.         0.         0.
  0.         1.        ]
 [

## (2) reinit 函数示例

In [85]:
from scipy import ndimage
import numpy as np

def reinit(sampling, struc):

    strucFull = np.zeros((nely + 2, nelx + 2))
    strucFull[1:-1, 1:-1] = struc

    dist_to_0 = ndimage.distance_transform_edt(strucFull, sampling=sampling)
    dist_to_1 = ndimage.distance_transform_edt(strucFull - 1, sampling=sampling)

    element_length = sampling / 2
    temp_0 = dist_to_0 - element_length
    temp_1 = dist_to_1 - element_length

    lsf = -(1 - strucFull) * temp_1 + strucFull * temp_0

    return lsf

# 测试参数
nelx, nely = 32, 20
domain = [0, 32, 0, 20]
ew = (domain[1]-domain[0]) / nelx 
eh = (domain[3]-domain[2]) / nely 

# 创建一个测试用的结构数组（含有一个孔洞）
struc = np.ones((nely, nelx))
struc[8:12, 13:19] = 0
strucFull = np.zeros((nely + 2, nelx + 2))
strucFull[1:-1, 1:-1] = struc
print("strucFull:", strucFull.shape, "\n", strucFull)

# 调用 reinit 函数
lsf = reinit(sampling=ew, struc=struc)
print("lsf:", lsf.shape, "\n", lsf.round(3))

# 可视化
from fealpy.mesh import UniformMesh2d
hx = (domain[1] - domain[0]) / nelx
hy = (domain[3] - domain[2]) / nely
mesh = UniformMesh2d(extent=(0, nelx+2, 0, nely+2), 
                    h=(hx, hy), origin=(domain[0], domain[2]))

import os
visualization_dir = 'visualization/'
os.makedirs(visualization_dir, exist_ok=True)
fname = os.path.join(visualization_dir, 'chaills_hole.vts')
mesh.to_vtk(filename=fname, 
                celldata={'strucFull': strucFull.flatten('F'),
                          'lsf': lsf.flatten('F')})

strucFull: (22, 34) 
 [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1. 1. 1. 1. 1. 1. 0.]
 [0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1. 1. 1. 1. 1. 1. 0.]
 [0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1. 1. 1. 1. 1. 1. 0.]
 [0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1. 1. 1. 1. 1. 1. 0.]
 [0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1. 1. 1. 1. 1. 1. 0.]
 [0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1. 1. 1. 1. 1. 1. 0.]
 [0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1. 1. 1. 1. 1. 1. 0.]
 [0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1. 1. 1. 1. 1. 1. 0.]
 [0. 1. 1. 1. 1

'visualization/chaills_hole.vts'

## (3) reinit 函数检验
符号距离函数
$$\phi(x,y) = \sqrt{(x-0.5)^2+(y-0.75)^2} - 0.15$$
其中$(x,y)\in[0, 1]\times[0,1]$。
$$|\nabla\phi| = \sqrt{\bigg(\frac{x-0.5}{\sqrt{(x-0.5)^2+(y-0.5)^2}}\bigg)^2 + \bigg(\frac{y-0.75}{\sqrt{(x-0.5)^2+(y-0.5)^2)}}\bigg)^2} = 1$$
非符号距离函数
$$\psi(x,y) = (x-0.5)^2+(y-0.75)^2 - 0.0115$$
其中$(x,y)\in[0, 1]\times[0,1]$。
$$|\nabla\psi| = \sqrt{(2(x-0.5)^2 + (2(y-0.75))^2} = 2\sqrt{(x-0.5)^2+(y-0.75)^2} = 2\sqrt{0.0115} \neq 1$$

In [86]:
import numpy as np
from fealpy.decorator import cartesian
from fealpy.mesh import UniformMesh2d
from scipy import ndimage

def reinit(sampling, struc):

    strucFull = np.zeros((nely + 2, nelx + 2))
    strucFull[1:-1, 1:-1] = struc

    dist_to_0 = ndimage.distance_transform_edt(strucFull, sampling=sampling)
    dist_to_1 = ndimage.distance_transform_edt(strucFull - 1, sampling=sampling)

    element_length = sampling / 2
    temp_0 = dist_to_0 - element_length
    temp_1 = dist_to_1 - element_length

    lsf = -(1 - strucFull) * temp_1 + strucFull * temp_0

    return lsf

# 非符号距离函数的水平集函数
@cartesian
def phi_nsd(p):
    x = p[..., 0]
    y = p[..., 1]
    val = (x - 0.5)**2 + (y - 0.75)**2 - 0.0115

    return val

# 符号距离函数的水平集函数
@cartesian
def phi_sd(p):
    x = p[..., 0]
    y = p[..., 1]
    val = np.sqrt((x - 0.5)**2 + (y - 0.75)**2) - 0.15

    return val

In [87]:
nelx, nely = 20, 20
domain = [0, 1, 0, 1]
hx = (domain[1] - domain[0]) / nelx
hy = (domain[3] - domain[2]) / nely
mesh = UniformMesh2d(extent=(0, nelx+2, 0, nely+2), 
                    h=(hx, hy), origin=(domain[0], domain[2]))

bc = mesh.entity_barycenter('cell')
phi_nsd_interpolate = phi_nsd(bc).reshape(nelx+2, nely+2).T
print("phi_nsd_interpolate:", phi_nsd_interpolate.shape, "\n", phi_nsd_interpolate)

phi_sd_interpolate = phi_sd(bc).reshape(nelx+2, nely+2).T

# 非符号距离函数对应的单元密度
strucFull_nsd = (phi_nsd_interpolate >= 0).astype(int)
struc_nsd = strucFull_nsd[1:-1, 1:-1]
print("struc_nsd:", struc_nsd.shape, "\n", struc_nsd)

# 符号距离函数对应的单元密度
strucFull_sd = (phi_sd_interpolate >= 0).astype(int)
struc_sd = strucFull_sd[1:-1, 1:-1]
print("struc_sd:", struc_sd.shape, "\n", struc_sd)


lsf_nsd_reinit = reinit(sampling=hx, struc=struc_nsd)
print("lsf_nsd_reinit:", lsf_nsd_reinit.shape, "\n", lsf_nsd_reinit.round(3))

strucFull_lsf_reinit = (lsf_nsd_reinit.flatten('F') >= 0).astype(int).reshape(nelx+2, nely+2).T
struc_lsf_reinit = strucFull_lsf_reinit[1:-1, 1:-1]
print("struc_lsf_reinit:", struc_lsf_reinit.shape, "\n", struc_lsf_reinit)


phi_nsd_interpolate: (22, 22) 
 [[ 7.3975e-01  6.9475e-01  6.5475e-01  6.1975e-01  5.8975e-01  5.6475e-01
   5.4475e-01  5.2975e-01  5.1975e-01  5.1475e-01  5.1475e-01  5.1975e-01
   5.2975e-01  5.4475e-01  5.6475e-01  5.8975e-01  6.1975e-01  6.5475e-01
   6.9475e-01  7.3975e-01  7.8975e-01  8.4475e-01]
 [ 6.6975e-01  6.2475e-01  5.8475e-01  5.4975e-01  5.1975e-01  4.9475e-01
   4.7475e-01  4.5975e-01  4.4975e-01  4.4475e-01  4.4475e-01  4.4975e-01
   4.5975e-01  4.7475e-01  4.9475e-01  5.1975e-01  5.4975e-01  5.8475e-01
   6.2475e-01  6.6975e-01  7.1975e-01  7.7475e-01]
 [ 6.0475e-01  5.5975e-01  5.1975e-01  4.8475e-01  4.5475e-01  4.2975e-01
   4.0975e-01  3.9475e-01  3.8475e-01  3.7975e-01  3.7975e-01  3.8475e-01
   3.9475e-01  4.0975e-01  4.2975e-01  4.5475e-01  4.8475e-01  5.1975e-01
   5.5975e-01  6.0475e-01  6.5475e-01  7.0975e-01]
 [ 5.4475e-01  4.9975e-01  4.5975e-01  4.2475e-01  3.9475e-01  3.6975e-01
   3.4975e-01  3.3475e-01  3.2475e-01  3.1975e-01  3.1975e-01  3.2475e-01
 

In [88]:
print("struc_error_nsd_sd:", np.sum(np.abs(struc_nsd - struc_sd)))
print("struc_error_sd_reinit:", np.sum(np.abs(struc_sd - struc_lsf_reinit)))
print("struc_error_nsd_reinit:", np.sum(np.abs(struc_nsd - struc_lsf_reinit)))

error_sd_nsd = np.max(np.abs(phi_nsd_interpolate - phi_sd_interpolate))
print("error_sd_nsd:", error_sd_nsd)
error_lsf_nsd = np.max(np.abs(lsf_nsd_reinit - phi_nsd_interpolate))
print("error_lsf_nsd:", error_lsf_nsd)
error_lsf_sd = np.max(np.abs(lsf_nsd_reinit - phi_sd_interpolate))
print("error_lsf_sd:", error_lsf_sd)

struc_error_nsd_sd: 16
struc_error_sd_reinit: 16
struc_error_nsd_reinit: 0
error_sd_nsd: 0.11148583103198026
error_lsf_nsd: 0.8904606781186548
error_lsf_sd: 0.8210484542847422


In [89]:
import os
visualization_dir = 'visualization/'
os.makedirs(visualization_dir, exist_ok=True)

fname = os.path.join(visualization_dir, 'chaills_reinit.vts')
mesh.to_vtk(filename=fname, 
                celldata={'strucFull_nsd': strucFull_nsd.flatten('F'),
                          'strucFull_sd': strucFull_sd.flatten('F'),
                          'phi_nsd': phi_nsd_interpolate.flatten('F'),
                          'phi_sd': phi_sd_interpolate.flatten('F'),
                          'strucFull_lsf_reinit': strucFull_lsf_reinit.flatten('F'),
                          'lsf_nsd_reinit': lsf_nsd_reinit.flatten('F')})

'visualization/chaills_reinit.vts'